In [23]:
from gensim import corpora
from collections import defaultdict
import re
import pandas as pd
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [10]:
documents = pd.read_pickle('gensim/preprocessed_text.pkl')

In [11]:
# remove words that only appear once in all documents
frequency = defaultdict(int)
for text in documents['text']:
    for token in text:
        frequency[token] += 1
        
documents['text'] = [[token for token in text if frequency[token] > 1]
              for text in documents['text']]

In [39]:
# still some web address tokens, so remove tokens containing 'http' or 'www'
documents_no_web = pd.DataFrame(columns=documents.columns)
for ind, row in documents.iterrows():
    text = row['text']
    new_text = []
    for word in text:
        if re.search(r'http', word):
            continue
        if re.search(r'www', word):
            continue
        new_text.append(word)
    row['text'] = new_text
    documents_no_web = documents_no_web.append(row)

In [40]:
# create dictionary
dictionary = corpora.Dictionary(documents_no_web['text'])
dictionary.save('gensim/dictionary.dict')

In [43]:
# convert to bag of words
corpus = [dictionary.doc2bow(text) for text in documents_no_web['text']]
corpora.MmCorpus.serialize('gensim/corpus.mm', corpus)